# Walkthrough on Hyperparameter Optimization using Weights and Biases

## Setting up the Working Directory
This cell is to ensure we change the directory to anomalib source code to have access to the datasets and config files. We assume that you already went through `001_getting_started.ipynb` and install the required packages.

In [1]:
import os
from pathlib import Path

from git.repo import Repo

current_directory = Path.cwd()
if current_directory.name == "300_benchmarking":
    # On the assumption that, the notebook is located in
    #   ~/anomalib/notebooks/300_benchmarking/
    root_directory = current_directory.parent.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory
else:
    # Otherwise, we'll need to clone the anomalib repo to the `current_directory`
    repo = Repo.clone_from(url="https://github.com/openvinotoolkit/anomalib.git", to_path=current_directory)
    root_directory = current_directory / "anomalib"

os.chdir(root_directory)

In [ ]:
%pip install .

> Note: Restart Runtime if promted by clicking the button at the end of the install logs

## Download and Setup Dataset

In [ ]:
!wget https://openvinotoolkit.github.io/anomalib/_downloads/3f2af1d7748194b18c2177a34c03a2c4/hazelnut_toy.zip
!unzip hazelnut_toy.zip -d datasets/ > /dev/null
!rm hazelnut_toy.zip

## Creating training configuration

Since we are using our [custom dataset](https://openvinotoolkit.github.io/anomalib/how_to_guides/train_custom_data.html) we need to modify the default configuration file. The following configuration is based on the one available here `anomalib/anomalib/models/stfpm/config.yaml`. We will first read the stfpm configs and replace the dataset configuration with our custom dataset configuration.

In [10]:
from omegaconf import OmegaConf

dataset_configuration_str = f"""
  name: hazelnut
  format: folder
  root: {str(root_directory / "datasets" / "hazelnut_toy")}
  normal_dir: good # name of the folder containing normal images.
  abnormal_dir: colour # name of the folder containing abnormal images.
  normal_test_dir: null # name of the folder containing normal test images.
  mask_dir: mask/colour # optional
  task: segmentation # classification or segmentation
  normalization: imagenet
  extensions: null
  split_ratio: 0.2  # ratio of the normal images that will be used to create a test split
  image_size: 256
  center_crop: 256
  train_batch_size: 32
  eval_batch_size: 32
  num_workers: 8
  transform_config:
    train: null
    eval: null
  test_split_mode: from_dir # options: [from_dir, synthetic]
  test_split_ratio: 0.2 # fraction of train images held out testing (usage depends on test_split_mode)
  val_split_mode: same_as_test # options: [same_as_test, from_test, synthetic]
  val_split_ratio: 0.5 # fraction of train/test images held out for validation (usage depends on val_split_mode)
  tiling:
    apply: false
    tile_size: null
    stride: null
    remove_border_count: 0
    use_random_tiling: False
    random_tile_count: 16
"""

config = OmegaConf.load(root_directory / "anomalib" / "models" / "stfpm" / "config.yaml")
dataset_configurations = OmegaConf.create(dataset_configuration_str)
config.dataset = dataset_configurations

## Create sweep configuration

The following configuration file is based on the one available at `anomalib/tools/hpo/configs/stfpm.yaml`. For this example we will use the STFPM model. It is possible to define multiple parameters for each section. For instance, we can define multiple image_sizes, or backbones. The sweep will then run for each combination of parameters. We will also define the number of runs for each combination of parameters with `observation_budget`. The sweep will run for a total of 10 runs for this config file. You could increase the `observation_budget` to run more runs. 

In [7]:
sweep_config_str = """
observation_budget: 10
method: bayes
metric:
  name: pixel_AUROC
  goal: maximize
parameters:
  dataset:
    category: capsule
    image_size:
      values:
      - 256    # Add as many values as you want.
  model:
    backbone:
      values:
      - resnet18    # Add as many backbones as you want.
    lr:
      min: 0.1
      max: 0.9
    momentum:
      min: 0.1
      max: 0.9
"""
sweep_config = OmegaConf.create(sweep_config_str)

> NOTE: To speed up the runs, we will use a small number of epochs and observation_budget. You could increase these parametere to get better results.

In [16]:
# NOTE: This is to speed up the runs. Increase these two get better results!
config.trainer.max_epochs = 5
sweep_config.observation_budget = 1

In [17]:
# Save the config files to the current directory.
OmegaConf.save(config, "model_config.yaml")
OmegaConf.save(sweep_config, "sweep_config.yaml")

In [18]:
!python ./tools/hpo/sweep.py --model stfpm --model_config model_config.yaml --sweep_config sweep_config.yaml

/home/sakcay/.pyenv/versions/3.8.13/envs/anomalib/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/sakcay/.pyenv/versions/3.8.13/envs/anomalib/lib/python3.8/site-packages/anomalib/config/config.py:238: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
/home/sakcay/.pyenv/versions/3.8.13/envs/anomalib/lib/python3.8/site-packages/anomalib/config/config.py:179: DeprecationWarning: The 'split_ratio' parameter is deprecated and will be removed in a future release. Please use 'test_split_ratio' instead.
  warn(
/home/sakcay/.pyenv/versions/3

While only a few parameters were shown in this example, you can call HPO on any of the parameters defined in the `model` and `dataset` section of the model configuration file.

Congratulations 🎉 You have now successfully optimized a model on your dataset.

To go into more detail, refer our documentation on [hyperparameter optimization](https://openvinotoolkit.github.io/anomalib/tutorials/hyperparameter_optimization.html).